In [1]:
#Import dependencies
import csv
import pandas as pd
import matplotlib.pyplot as plt
import gmaps
import gmaps.datasets
import requests
import json
from config import gkey 
from random import sample

In [2]:
#Load CSV
accident_data = pd.read_csv("Resources/2021_us_accidents.csv")
accident_data.head()

,Severity,Start_Lat,Start_Lng,State,Roundabout,Stop,Traffic_Signal,Sunrise_Sunset,Visibility(mi),Crossing
0,2,42.382359,-71.022318,MA,False,False,False,Night,10.0,False
1,2,40.908676,-123.707116,CA,False,False,False,Night,2.0,False
2,2,38.970628,-77.116333,MD,False,False,False,Day,10.0,False
3,2,47.642651,-122.318590,WA,False,False,False,Day,10.0,False
4,2,37.419536,-122.090640,CA,False,False,False,Day,10.0,False


In [3]:
#Add a column to count each accident as 1
accident_data["# of accidents"] = 1
accident_data.head()

,Severity,Start_Lat,Start_Lng,State,Roundabout,Stop,Traffic_Signal,Sunrise_Sunset,Visibility(mi),Crossing,# of accidents
0,2,42.382359,-71.022318,MA,False,False,False,Night,10.0,False,1
1,2,40.908676,-123.707116,CA,False,False,False,Night,2.0,False,1
2,2,38.970628,-77.116333,MD,False,False,False,Day,10.0,False,1
3,2,47.642651,-122.318590,WA,False,False,False,Day,10.0,False,1
4,2,37.419536,-122.090640,CA,False,False,False,Day,10.0,False,1


In [4]:
#See how many rows there are
len(accident_data["State"])

1511745

In [5]:
#Pull out columns for the heatmap
accident_data[["Start_Lat","Start_Lng","State","Severity"]][724:737]

,Start_Lat,Start_Lng,State,Severity
724,34.404527,-117.567234,CA,2
725,38.068042,-77.330525,VA,4
726,25.659734,-80.416025,FL,2
727,25.657368,-80.415920,FL,2
728,40.587741,-111.922077,UT,2
729,26.736497,-80.090694,FL,2
730,33.773968,-117.818345,CA,2
731,25.733210,-80.307124,FL,2
732,25.733395,-80.301272,FL,2
733,25.733257,-80.305571,FL,2


In [14]:
#Create a sample for the limitations of our API key
sample = accident_data.sample(n = 10 , replace = False)
print(sample)

         Severity  Start_Lat   Start_Lng State  Roundabout   Stop  \
535245          2  33.925529 -118.309155    CA       False  False   
1490942         2  36.649307 -121.726251    CA       False  False   
337382          2  36.834370  -76.109871    VA       False  False   
298232          2  47.325295  -95.966308    MN       False  False   
502989          2  30.189952  -85.789679    FL       False  False   
1341467         2  37.776271 -121.319005    CA       False  False   
1328677         2  40.900441 -111.926002    UT       False  False   
1217865         2  25.696934  -80.333826    FL       False  False   
175016          2  30.313305  -87.490357    FL       False  False   
128362          2  35.081394  -81.624256    SC       False  False   

         Traffic_Signal Sunrise_Sunset  Visibility(mi)  Crossing  \
535245            False            Day            10.0     False   
1490942           False            Day            10.0     False   
337382            False          Nig

In [7]:
#Data for just Minnesota
MN_accident_data = accident_data.loc[accident_data["State"] == "MN"]
MN_accident_data

,Severity,Start_Lat,Start_Lng,State,Roundabout,Stop,Traffic_Signal,Sunrise_Sunset,Visibility(mi),Crossing,# of accidents
14,2,45.067409,-93.185854,MN,False,False,False,Day,10.0,False,1
100,2,44.965755,-93.279326,MN,False,False,False,Night,10.0,False,1
101,2,44.965009,-93.287549,MN,False,False,False,Day,10.0,False,1
102,2,44.965656,-93.288114,MN,False,False,False,Day,10.0,True,1
103,2,44.985306,-93.288355,MN,False,False,False,Day,10.0,False,1
...,...,...,...,...,...,...,...,...,...,...,...
1511590,2,45.019767,-93.188857,MN,False,False,False,Night,10.0,False,1
1511646,2,45.069481,-93.293681,MN,False,False,False,Night,8.0,False,1
1511676,2,44.966642,-93.224599,MN,False,False,False,Night,7.0,False,1
1511697,2,44.859870,-93.386331,MN,False,False,False,Night,6.0,False,1


In [8]:
ustates = []
for x in accident_data["State"]:
    if x not in ustates:
        ustates.append(x)
print(ustates)
#This list includes the continential US and Washington DC. (Excludes AK and HI)

['MA', 'CA', 'MD', 'WA', 'NC', 'FL', 'MO', 'TX', 'MN', 'AZ', 'AL', 'NJ', 'VA', 'SC', 'UT', 'TN', 'CO', 'NY', 'LA', 'PA', 'OR', 'WV', 'IL', 'AR', 'MT', 'DC', 'MI', 'NM', 'CT', 'MS', 'ND', 'IN', 'ID', 'GA', 'OH', 'NV', 'OK', 'DE', 'WY', 'KY', 'NH', 'NE', 'KS', 'IA', 'SD', 'RI', 'WI', 'ME', 'VT']


In [9]:
# url = f"https://maps.googleapis.com/maps/api/geocode/json?key={gkey}&address="
# target_url = f"https://maps.googleapis.com/maps/api/geocode/json?{target_city}"
# target_city = accident_data["City"]+ ", " +accident_data["State"]

# geo_data = requests.get(target_url).json


In [15]:
#Code for heatmap
gmaps_key = gmaps.configure(api_key = gkey)
locations = sample[["Start_Lat","Start_Lng"]]
weights = sample["# of accidents"][0:10]

In [16]:
#Code to display heatmap
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights = weights[0:10]))
fig

Figure(layout=FigureLayout(height='420px'))